In [ ]:
cd "/content/drive/My Drive/"

/content/drive/My Drive


In [ ]:
!pip install fasttext

     |████████████████████████████████| 71kB 1.9MB/s 
  Created wheel for fasttext: filename=fasttext-0.9.2-cp36-cp36m-linux_x86_64.whl size=3019120 sha256=62bae151891073e56f8f796d95fdfe2051b5b2185e2ead1852c55c8492a5995e
  Stored in directory: /root/.cache/pip/wheels/98/ba/7f/b154944a1cf5a8cee91c154b75231136cc3a3321ab0e30f592
Successfully built fasttext


In [ ]:
import pandas as pd
from sklearn.utils import resample
import csv
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
from bs4 import BeautifulSoup
import string
import re
import fasttext

nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
data = pd.read_csv("sentimentMemes.csv")

In [ ]:
data['overall_sentiment'].value_counts()

positive         3127
neutral          2201
very_positive    1033
negative          480
very_negative     151
Name: overall_sentiment, dtype: int64

In [ ]:
data['overall_sentiment'] = data['overall_sentiment'].replace({'very_positive': 'positive', 'very_negative': 'negative'})

In [ ]:
data['overall_sentiment'].value_counts()

positive    4160
neutral     2201
negative     631
Name: overall_sentiment, dtype: int64

In [ ]:
classes = data['overall_sentiment'].unique()
print(classes)

['positive' 'neutral' 'negative']


In [ ]:
positive = data[data.overall_sentiment==classes[0]]
neutral = data[data.overall_sentiment==classes[1]]
negative = data[data.overall_sentiment==classes[2]]

In [ ]:
neutral_upsampled = resample(neutral,
                          replace=True,
                          n_samples=len(positive), 
                          random_state=27)
negative_upsampled = resample(negative,
                          replace=True,
                          n_samples=len(positive), 
                          random_state=27)

In [ ]:
upsampled = [positive, neutral_upsampled,negative_upsampled]
upsampled = pd.concat(upsampled)

In [ ]:
upsampled['overall_sentiment'].value_counts()

positive    4160
neutral     4160
negative    4160
Name: overall_sentiment, dtype: int64

In [ ]:
def clean_data(text):
  text = str(text)
  no_punct = "".join([c for c in text if c not in string.punctuation])
  url_pattern = r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
  text = re.sub(url_pattern, ' ', no_punct)
  text = ''.join(character for character in text if ord(character) < 128)
  emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
  text = emoji_pattern.sub(r'', text)
  toknizer = RegexpTokenizer(r'\w+')
  text = toknizer.tokenize(text.lower())
  text = [w for w in text if w not in stopwords.words('english')]
  lemmatizer = WordNetLemmatizer()
  text = [lemmatizer.lemmatize(i) for i in text]
  stemmer = PorterStemmer()
  text = " ".join([stemmer.stem(i) for i in  text])
  return text

In [ ]:
upsampled['text_corrected'] = upsampled['text_corrected'].map(lambda x: clean_data(x))

In [ ]:
upsampled['text_corrected']

0       look friend lightyear sohalikut trend play 10 ...
1       best 10 yearchalleng complet le 4 year kudu na...
2       sam thorn strippin follow follow saw everyon p...
3                         10 year challeng sweet dee edit
8       fornit die 10 month minecraft never die 10 yea...
                              ...                        
4229            your go start world war 3 presid wont let
185     condit extra class facebookcomcompleteg comple...
4801    technic hillari clinton turn point usa part 39...
4221                                       donald clinton
10      probabl first man 10 year challeng chennai pas...
Name: text_corrected, Length: 12480, dtype: object

In [ ]:
df1 = upsampled[upsampled['text_corrected'].str.contains("penguin meme close close fling imgflipcom entir sub smile wave")] 
print(df1)

          image_name  ... overall_sentiment
3114  image_3115.png  ...          positive

[1 rows x 4 columns]


In [ ]:
col = ['overall_sentiment','text_corrected']
train = upsampled[col]
train['overall_sentiment']=['__label__'+ s for s in train['overall_sentiment']]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
train

,overall_sentiment,text_corrected
0,__label__positive,look friend lightyear sohalikut trend play 10 ...
1,__label__positive,best 10 yearchalleng complet le 4 year kudu na...
2,__label__positive,sam thorn strippin follow follow saw everyon p...
3,__label__positive,10 year challeng sweet dee edit
8,__label__positive,fornit die 10 month minecraft never die 10 yea...
...,...,...
4229,__label__negative,your go start world war 3 presid wont let
185,__label__negative,condit extra class facebookcomcompleteg comple...
4801,__label__negative,technic hillari clinton turn point usa part 39...
4221,__label__negative,donald clinton


In [ ]:
train = train.sample(n=len(train), random_state=42)

In [ ]:
train_data = train[:int(len(train) * 70 / 100)]
test_data = train[int(len(train) * 70 / 100):]

In [ ]:
train_data.to_csv(r'/content/drive/My Drive/train.txt', index=False, sep=' ', header=False, quoting=csv.QUOTE_NONE, quotechar="", escapechar=" ")
test_data.to_csv(r'/content/drive/My Drive/test.txt', index=False, sep=' ', header=False, quoting=csv.QUOTE_NONE, quotechar="", escapechar=" ")

In [ ]:
model = fasttext.train_supervised(input = '/content/drive/My Drive/train.txt', lr=0.1, epoch=50, wordNgrams=2, loss='hs')

In [ ]:
model.save_model("sentimental_analysis/fasttext_model_zero.ftz")

In [ ]:
model = fasttext.load_model("sentimental_analysis/fasttext_model_zero.bin")

In [ ]:
print(model.words)
print(model.labels)

['</s>', 'meme', 'like', 'bill', 'im', 'imgflipcom', 'dont', 'get', 'one', 'make', 'your', 'look', 'quickmemecom', 'say', 'friend', 'know', 'go', 'peopl', 'see', 'want', 'time', 'think', 'memecentercom', 'woman', 'memegeneratornet', 'girl', 'love', 'never', 'u', 'tell', 'call', 'someon', 'trump', 'come', 'doesnt', 'made', 'fuck', 'kill', 'got', 'hate', 'year', 'cant', 'day', 'countri', 'need', 'take', 'find', 'cat', 'best', 'alway', 'hillari', 'girlfriend', 'would', 'life', 'man', 'thing', 'ask', 'back', 'still', 'world', 'hitler', 'new', 'good', 'guy', 'liam', 'that', 'first', 'oh', 'funni', 'use', 'white', 'tri', 'work', 'give', 'everyon', 'smart', 'better', 'stop', 'kid', 'post', 'let', 'keep', 'watch', 'hey', 'mr', 'could', 'put', 'mean', 'obama', 'shit', 'memecent', 'presid', 'person', 'without', 'racist', 'face', 'old', 'said', 'even', 'feel', 'real', 'actual', 'chuck', 'norri', 'gonna', 'told', 'follow', 'start', 'last', 'die', 'steve', 'much', 'job', 'ever', 'show', 'name', 'mo

In [ ]:
def print_results(N, p, r):
    print("N\t" + str(N))
    print("P@{}\t{:.3f}".format(1, p))
    print("R@{}\t{:.3f}".format(1, r))

In [ ]:
print_results(*model.test('/content/drive/My Drive/test.txt'))

N	3744
P@1	0.794
R@1	0.794


In [ ]:
model.predict("hey girl keep law bodi keep hand")

(('__label__negative',), array([0.9984532]))